In [16]:
from typing import List
import pandas as pd
import numpy as np

In [22]:
# Example input data 
data = pd.DataFrame({
    'nx': [1, 4] + [5] * 17,  # Длительность возрастных интервалов
    'Mx': [
        0.0039, 0.0003, 0.0001, 0.0002, 0.0004, 0.0005, 0.0006, 0.0011,
        0.0018, 0.0028, 0.0037, 0.0048, 0.0068, 0.0101, 0.0156, 0.0254,
        0.0414, 0.0836, 0.1856  # Сила смертности
    ]
})

def life_table(data: pd.DataFrame) -> pd.DataFrame:
    """
    Construct a life table from the dataframe, containig columns with population age 
    interval (nx) and raw death rate (Mx) in a certain year

    Parameters:
    -----------
    data: pd.DataFrame
        A dataframe containig two columns described below
    nx: List[int]
        A list containing the lengths of age intervals for the obtained population group
    Mx: List[float]
        A list containing raw mortality rate for population group within given age intervals
    
    Returns:
    --------
    pd.DataFrame
        A DataFrame representing the life table with the following columns:

        - Age: Age group label (e.g., '0', '1-4', '5-9', ..., '85+')
        - nx: Duration of the age interval in years
        - Mx: Mortality rate for the interval
        - qx: Probability of dying within the age interval (age-specific mortality rate)
        - lx: Number of individuals surviving to the start of the age interval
                  (initial cohort size is typically set to 100,000)
        - dx: Number of deaths during the age interval
        - Lx: Person-years lived in the age interval
        - Tx: Total person-years lived after the start of age x
        - ex: Life expectancy at the beginning of the age interval
    
    Notes:
    ------
    - For the open-ended interval, qx is assumed to be 1
    - Lx for the open interval is calculated as lx / Mx
    """
    n = len(data)
    qx = np.zeros(n)  
    lx = np.zeros(n)  
    dx = np.zeros(n)  
    Lx = np.zeros(n)  
    Tx = np.zeros(n)  
    ex = np.zeros(n)

    lx[0] = 10**6
    for _ in range(n):
        if _ < n - 1:
            qx[_] = 1 - np.exp(-data['Mx'].iloc[_] * data['nx'].iloc[_])
        else:
            qx[_] = 1.0
        dx[_] = lx[_] * qx[_]

        if _ == n - 1:
            Lx[_] = lx[_] / data['Mx'].iloc[_] if data['Mx'].iloc[_] != 0 else 0 
        else:
            Lx[_] = data['nx'].iloc[_] * lx[_] - (data['nx'].iloc[_] / 2) * dx[_]
            lx[_ + 1] = lx[_] - dx[_]

    Tx[-1] = Lx[-1]
    ex[-1] = Tx[-1] / lx[-1] if lx[-1] != 0 else 0

    for _ in range(n - 2, -1, -1):
        Tx[_] = Lx[_] + Tx[_ + 1]
        ex[_] = Tx[_] / lx[_] if lx[_] != 0 else 0
    
    # Для примера (надо будет доработать)
    age_groups = ["0", "1-4", "5-9", "10-14", "15-19", "20-24", "25-29", "30-34",
                  "35-39", "40-44", "45-49", "50-54", "55-59", "60-64", "65-69",
                  "70-74", "75-79", "80-84", "85+"]

    # Создание DataFrame
    life_table = pd.DataFrame({
        'Age': age_groups[:n],
        'nx': data['nx'],
        'Mx': data['Mx'],
        'qx': np.round(qx, 5),
        'lx': np.round(lx).astype(int),
        'dx': np.round(dx).astype(int),
        'Lx': np.round(Lx).astype(int),
        'Tx': np.round(Tx).astype(int),
        'ex': np.round(ex, 2)
    })

    return life_table

In [23]:
life_table(data)

,Age,nx,Mx,qx,lx,dx,Lx,Tx,ex
0,0,1,0.0039,0.00389,1000000,3892,998054,77785525,77.79
1,1-4,4,0.0003,0.00120,996108,1195,3982041,76787471,77.09
2,5-9,5,0.0001,0.00050,994913,497,4973322,72805430,73.18
3,10-14,5,0.0002,0.00100,994416,994,4969593,67832108,68.21
4,15-19,5,0.0004,0.00200,993422,1985,4962147,62862515,63.28
5,20-24,5,0.0005,0.00250,991437,2475,4950996,57900368,58.40
6,25-29,5,0.0006,0.00300,988961,2962,4937401,52949373,53.54
7,30-34,5,0.0011,0.00548,985999,5408,4916474,48011972,48.69
8,35-39,5,0.0018,0.00896,980591,8786,4880990,43095497,43.95
9,40-44,5,0.0028,0.01390,971805,13510,4825249,38214508,39.32
